<a href="https://colab.research.google.com/github/UtkarshAIITB/IPL-Data-Analysis/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [102]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
new = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/new.csv')
# new.head()

In [104]:
t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/t.csv')
# t.head()

In [105]:
t.drop(columns = ['Unnamed: 0'], inplace = True)
t.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,597998,2013-04-03,9,6,9,9,1,0.1,6,9,2013,1,0,0,128,0,1
1,597998,2013-04-03,9,6,9,9,1,0.2,6,9,2013,1,0,0,128,4,1
2,597998,2013-04-03,9,6,9,9,1,0.3,6,9,2013,1,0,0,128,5,1
3,597998,2013-04-03,9,6,9,9,1,0.4,6,9,2013,1,0,0,128,5,1
4,597998,2013-04-03,9,6,9,9,1,0.5,6,9,2013,1,0,0,128,5,1


In [106]:
new.drop(columns = ['Unnamed: 0'], inplace = True)

In [107]:
new.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,0.1,9,5,2008,1,0,0,222,1,0
1,335982,2008-04-18,5,9,5,9,1,0.2,9,5,2008,1,0,0,222,1,0
2,335982,2008-04-18,5,9,5,9,1,0.3,9,5,2008,1,0,0,222,2,0
3,335982,2008-04-18,5,9,5,9,1,0.4,9,5,2008,1,0,0,222,2,0
4,335982,2008-04-18,5,9,5,9,1,0.5,9,5,2008,1,0,0,222,2,0


In [108]:
# new is the training set and t is the test set

In [109]:
tnew = t.drop(t[(t.over <= 14.0) | (t.over >=16.0)].index)
tnew.reset_index(drop = True)
y1test = tnew['total_runs_y']
y2test = tnew['winner']
tnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
tnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,9,6,9,1,14.1,6,9,1,0,0,93,5
1,9,6,9,1,14.2,6,9,1,0,0,94,5
2,9,6,9,1,14.3,6,9,1,0,0,94,5
3,9,6,9,1,14.4,6,9,1,0,0,94,5
4,9,6,9,1,14.5,6,9,1,0,0,95,5


In [110]:
xnew = new.drop(new[(new.over <=14.0) | (new.over >=16.0)].index)
xnew.reset_index(drop = True)
y1train = xnew['total_runs_y']
y2train = xnew['winner']
xnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
xnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,5,9,5,1,14.1,9,5,1,0,0,134,2
1,5,9,5,1,14.2,9,5,1,0,0,135,2
2,5,9,5,1,14.3,9,5,1,0,0,141,2
3,5,9,5,1,14.4,9,5,1,0,0,145,2
4,5,9,5,1,14.5,9,5,1,0,0,147,2


In [111]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [112]:
xnewt = sc.fit_transform(xnew) 
tnewt = sc.fit_transform(tnew)

In [113]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [184]:
model = Sequential()
model.add(Dense(200, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(100, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(50, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [185]:
model.fit(x=xnewt, y=y1train, epochs=300, 
          validation_data=(tnewt,y1test))

Epoch 1/300
540/540 [==============================] - 2s 2ms/step - loss: 4083.1348 - val_loss: 359.6973
Epoch 2/300
540/540 [==============================] - 1s 2ms/step - loss: 1590.6937 - val_loss: 264.9135
Epoch 3/300
540/540 [==============================] - 1s 2ms/step - loss: 1486.4733 - val_loss: 251.6593
Epoch 4/300
540/540 [==============================] - 1s 2ms/step - loss: 1465.0397 - val_loss: 245.2000
Epoch 5/300
540/540 [==============================] - 1s 2ms/step - loss: 1423.4463 - val_loss: 340.7914
Epoch 6/300
540/540 [==============================] - 1s 2ms/step - loss: 1385.7841 - val_loss: 218.0543
Epoch 7/300
540/540 [==============================] - 1s 2ms/step - loss: 1335.3812 - val_loss: 234.1846
Epoch 8/300
540/540 [==============================] - 1s 2ms/step - loss: 1337.3740 - val_loss: 192.2826
Epoch 9/300
540/540 [==============================] - 1s 2ms/step - loss: 1280.6667 - val_loss: 200.5973
Epoch 10/300
540/540 [========================

In [186]:
y1test = y1test.ravel()

In [187]:
predictions = model.predict(tnewt)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=y1test
sample.head(10)

,Predict,Actual
0,140.770950,128
1,141.463867,128
2,140.392212,128
3,139.454056,128
4,139.960083,128
5,140.518951,128
6,137.946701,128
7,134.346832,128
8,134.515656,128
9,134.409332,128


In [188]:
def custom_accuracy(y_test,y_pred,thresold):
    right = 0
    l = len(y_pred)
    for i in range(0,l):
        if(abs(y_pred[i]-y_test[i]) <= thresold):
            right += 1
    return ((right/l)*100)

In [189]:
custom_accuracy(sample['Actual'] , sample['Predict'],10)

42.168674698795186

In [190]:
# sample.to_csv('sample.csv')

In [191]:
# try1 = sample[sample.Actual == 128]
# try1.head()

In [192]:
# try1.shape[0]

In [193]:
# try1.sum()

In [194]:
# (try1.sum())/try1.shape[0]

In [195]:
# try2 = sample[sample.Actual == 129]
# (try2.sum())/try2.shape[0]

In [196]:
# try2 = sample[sample.Actual == 156]
# result = (try2.sum())/try2.shape[0]

In [197]:
# result

In [198]:
# res2 = result.values.reshape(1,2)

In [199]:
# res2

In [200]:
# res2.shape

In [201]:
# df.loc[len(df)] = your_array
# results.loc[len(results)] = res2
# df.append(pd.DataFrame(arr).T)
# results = results.append(pd.DataFrame(res2))
# results.append(pd.DataFrame(res3))

In [202]:
ar1 = sample['Actual'].unique()

In [203]:
results = pd.DataFrame()

In [204]:
for items in ar1:
  # print(items)
  try2 = sample[sample.Actual == items]
  result = (try2.sum())/try2.shape[0]
  # print(result)
  res = result.values.reshape(1,2)
  # print(res)
  results = results.append(pd.DataFrame(res))

In [205]:
results.head()

,0,1
0,137.686096,128.0
0,143.382920,129.0
0,152.659762,156.0
0,154.747396,154.0
0,139.486552,126.0


In [206]:
# rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)
results.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)

In [207]:
results.reset_index(drop = True)

,Predict,Actual
0,137.686096,128.0
1,143.382920,129.0
2,152.659762,156.0
3,154.747396,154.0
4,139.486552,126.0
...,...,...
77,190.851249,190.0
78,124.666229,113.0
79,157.505296,172.0
80,153.803608,134.0


In [208]:
results.tail()

,Predict,Actual
0,190.851249,190.0
0,124.666229,113.0
0,157.505296,172.0
0,153.803608,134.0
0,171.700704,192.0


In [209]:
print(custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))

56.09756097560976


In [210]:
# results.info()

In [211]:
# results['Actual']

take epochs = 90 next time

In [212]:
tests = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/testset.csv')
# tests.head()
tests.drop(columns = ['Unnamed: 0'], inplace = True)
tests.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.1,12.0,5.0,1,0,0,159,2,0
1,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.2,12.0,5.0,1,0,0,159,2,0
2,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.3,12.0,5.0,1,0,0,159,2,0
3,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.4,12.0,5.0,1,0,0,159,4,0
4,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.5,12.0,5.0,1,0,0,159,4,0


In [213]:
sets = tests.drop(tests[(tests.over <= 14.0) | (tests.over >=16.0)].index)
sets.reset_index(drop = True)
yt1 = sets['total_runs_y']
yt2 = sets['winner']
sets.drop(columns = ['id', 'date', 'total_runs_y', 'winner'], inplace = True)
sets.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,12.0,5.0,5.0,1,14.1,12.0,5.0,1,0,0,120,3
1,12.0,5.0,5.0,1,14.2,12.0,5.0,1,0,0,124,3
2,12.0,5.0,5.0,1,14.3,12.0,5.0,1,0,0,124,3
3,12.0,5.0,5.0,1,14.4,12.0,5.0,1,0,0,125,3
4,12.0,5.0,5.0,1,14.5,12.0,5.0,1,0,0,127,3


In [214]:
yt1 = yt1.ravel()

In [215]:
xs = sc.fit_transform(sets)

In [216]:
predictions = model.predict(xs)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=yt1
sample.head(10)

,Predict,Actual
0,171.881622,159
1,173.432800,159
2,173.143127,159
3,173.317871,159
4,173.932800,159
5,174.107025,159
6,172.239899,159
7,172.393066,159
8,173.053741,159
9,174.904449,159


In [217]:
ar1 = sample['Actual'].unique()
results = pd.DataFrame()
for items in ar1:
  # print(items)
  try2 = sample[sample.Actual == items]
  result = (try2.sum())/try2.shape[0]
  # print(result)
  res = result.values.reshape(1,2)
  # print(res)
  results = results.append(pd.DataFrame(res))

In [218]:
results.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)
results.reset_index(drop = True)
print(custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))

37.142857142857146
